In [1]:
import pandas as pd
import glob
import time

path = '../data/raw/*.csv'
csv_files = glob.glob(path)
df = []

for file in csv_files:
    df = pd.read_csv(file)
    print(list(df))

['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price']
['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price']
['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'remaining_lease', 'resale_price']
['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'remaining_lease', 'resale_price']
['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model', 'lease_commence_date', 'resale_price']


In [16]:
def numbers_only(string):
    year_index = string.index(' y')
    year = float(string[:year_index])
    duration = 0
    if ' months' in string:
        month_index_start = string.index('s ') + 2
        month_index_end = string.index(' m')
        month = string[month_index_start:month_index_end]
        month_adjusted = float(month) / 12
        duration = year + month_adjusted
    else:
        duration = year
    return duration 

i = 0

for file in csv_files:
    i += 1
    df = pd.read_csv(file)

#print(df[:5])
        
    print(len(df))
    df.drop_duplicates()
    print(len(df))
    
    start = time.time()
    df['month'] = df['month'].apply(lambda x: pd.to_datetime(x + '-01'))
    df.sort_values(by='month')

    elapsed = time.time() - start
    print(f'took {elapsed} seconds to run')

    if 'remaining_lease' in df.columns:
        try:
            df['remaining_lease'] = df['remaining_lease'].astype('int32')
        except ValueError:
            df['remaining_lease'] = df['remaining_lease'].apply(lambda x: numbers_only(x) if type(x) == str else None)
    else:
        df.insert(9, 'remaining_lease', 99 - (df['month'].dt.year - df['lease_commence_date']))

    for column in df.columns:
        exclude = ['month', 'floor_area_sqm', 'lease_commence_date', 'resale_price', 'remaining_lease']
        if column not in exclude:
            df[column] = df[column].str.replace('-', ' ').str.lower()
            df[column] = df[column].astype('category')

    i_string = str(i)
    df.to_csv(f'../data/cleaned data {i_string}.csv', index=False)

52203
52203
took 16.475518703460693 seconds to run
369651
369651
took 64.06027460098267 seconds to run
195062
195062
took 71.21889996528625 seconds to run
37153
37153
took 21.2409086227417 seconds to run
287196
287196
took 157.30704402923584 seconds to run


In [6]:
import psycopg2
import yaml

def load_config():
    with open('../config.yaml', 'r') as file:
        config = yaml.safe_load(file)
    return config

config = load_config()

database_config = config['database']
dbname = database_config['name']
user = database_config['user']
password = database_config['password']
host = database_config['host']

print(f'Connecting to {dbname} at {host} with user {user}')

Connecting to symmetrical telegram at localhost with user postgres


In [24]:
# Create a connection to PostgreSQL
try:
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
    print(f'Connected to PostgreSQL')
except psycopg2.Error as e:
    print(f'Error occurred: {e}')
    sys.exit(1) # Indicate error occurred during execution

# Create a cursor object
cursor = conn.cursor()

Connected to PostgreSQL


In [25]:
cursor.execute("DROP TABLE IF EXISTS flats;")
cursor.execute("DROP TABLE IF EXISTS lease;")

In [26]:
# Create the table if it doesn't already exist
create_table_query = """
CREATE TABLE IF NOT EXISTS flats (
    flat_id SERIAL PRIMARY KEY,
    commence_date INTEGER NOT NULL,
    remaining_lease FLOAT,
    town VARCHAR(100) NOT NULL,
    street_name VARCHAR(255) NOT NULL,
    block VARCHAR(10) NOT NULL,
    flat_type VARCHAR(50) NOT NULL,
    flat_model VARCHAR(50) NOT NULL,
    storey_range VARCHAR(50) NOT NULL,
    floor_area_sqm FLOAT NOT NULL
);

CREATE TABLE IF NOT EXISTS lease (
    lease_id SERIAL PRIMARY KEY,
    flat_id INTEGER NOT NULL,
    month TIMESTAMP NOT NULL,
    resale_price FLOAT NOT NULL,
    CONSTRAINT fk_flats FOREIGN KEY (flat_id) REFERENCES flats(flat_id)
    ON DELETE CASCADE
    ON UPDATE CASCADE
);
"""

try:
    cursor.execute(create_table_query)
    conn.commit()  # Commit the creation of the table
    print('Successfully created flats and lease tables')
except psycopg2.Error as e:
    print(f'Error occurred: {e}')
    conn.rollback() # Rollback transaction to prevent partial changes from being kept
    cursor.close() # Release resources
    conn.close() # Prevent connection leaks

Successfully created flats and lease tables


In [27]:
path1 = '../data/*.csv'
csvfiles = glob.glob(path1)
df1 = []

def validate_row(row):
    mandatory_fields = ['lease_commence_date', 'town', 'street_name', 'block', 'flat_type', 'flat_model', 'storey_range', 'floor_area_sqm', 'month', 
                        'resale_price']
    for field in mandatory_fields:
        if pd.isnull(row[field]):
            return False
    return True

for file in csvfiles:
    start1 = time.time()
    df1 = pd.read_csv(file)

    # Create a list to store values for batch insertion
    flat_values = []
    lease_values = []

    for index, row in df1.iterrows():
        if not validate_row(row):
            print(f'Skipping row {index} in file {file} due to missing mandatory information')
            continue

        # Prepare flat data for batch insert
        flat_values.append((row['lease_commence_date'], row['town'], row['street_name'], row['block'], row['flat_type'], row['flat_model'], 
                            row['storey_range'], row['floor_area_sqm'], row['remaining_lease']))

        # Prepare lease data for batch insert
        lease_values.append((row['town'], row['street_name'], row['month'], row['resale_price']))

    try:
        # Bulk insert into flats table
        insert_flats_query = """
            INSERT INTO flats (commence_date, town, street_name, block, flat_type, flat_model, storey_range, floor_area_sqm, remaining_lease)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.executemany(insert_flats_query, flat_values)

        # Commit after bulk insert
        conn.commit()

        # Retrieve flat_ids for the inserted flats in one query
        pairs_set = set()
        for row in lease_values:
            town = row[0]
            street_name = row[1]
            pairs_set.add((town, street_name))
        pairs = tuple(pairs_set) # A tuple of tuples

        cursor.execute("""
            SELECT flat_id, town, street_name FROM flats WHERE (town, street_name) IN %s
        """, (pairs,)) # Pass pairs as a single argument
        flat_ids = {f"{town}-{street_name}": flat_id for flat_id, town, street_name in cursor.fetchall()}

        # Now prepare lease data using the fetched flat_ids
        lease_values = [(flat_ids[f"{town}-{street_name}"], month, resale_price) for town, street_name, month, resale_price in lease_values]

        # Bulk insert into lease table
        insert_lease_query = """
            INSERT INTO lease (flat_id, month, resale_price)
            VALUES (%s, %s, %s)
        """
        cursor.executemany(insert_lease_query, lease_values)

        # Commit after bulk insert into lease table
        conn.commit()

        elapsed1 = time.time() - start1
        print(f'Successfully processed file {file} in {elapsed} seconds')
    
    except psycopg2.Error as e:
        print(f'Error processing file {file}: {e}')
        conn.rollback()
        continue

# Close the cursor and connection
cursor.close()
conn.close()

Successfully processed file ../data\cleaned data 1.csv in 157.30704402923584 seconds
Successfully processed file ../data\cleaned data 2.csv in 157.30704402923584 seconds
Successfully processed file ../data\cleaned data 3.csv in 157.30704402923584 seconds
Successfully processed file ../data\cleaned data 4.csv in 157.30704402923584 seconds
Successfully processed file ../data\cleaned data 5.csv in 157.30704402923584 seconds


In [28]:
# Close the cursor and connection
cursor.close()
conn.close()
print ('Closed connection')

Closed connection
